In [1]:
# Access parent directories
from sys import path
from os.path import abspath
path += [abspath("../"*i) for i in (1, 2, 3)]

In [2]:
from QLab import Environment, SimpleAgent, QNetwork
import torch
from torch import nn
import torch.nn.functional as F

class DeepNetwork(QNetwork):
	"Simple and probably terrible triple layered qnetwork."
	def __init__(self, h, w):
		super().__init__()
		self.conv1 = nn.Conv2d(4, 16, kernel_size=5, stride=2)
		self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=2)
		self.connected1 = nn.Linear(32 * 4 * 5, 128)
		self.connected2 = nn.Linear(128, 4)

	def forward(self, x):
		x = F.relu(self.conv1(x))
		x = F.relu((self.conv2(x)))
		x = torch.flatten(x, 1)
		x = F.relu(self.connected1(x))
		x = F.relu(self.connected2(x))
		return x

class RescaledRewardsAgent(SimpleAgent):
	def feedback(self, state, action, reward, new_state):
		reward += 1
		if reward < 0: reward *= 0
		reward *= 1000000
		self.memory.push(state, action, reward, new_state)
		self.optimizer(gamma = self.gamma)

	def __call__(self, state):
		#print(self.policy_net(state.unsqueeze(0)))
		super().__call__(state)

height, width = Environment().shape
shnet = DeepNetwork(height, width)
agent = RescaledRewardsAgent(shnet, epsilon=0.9)

pygame 2.1.2 (SDL 2.0.16, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
agent.train(episodes=100, update_frq=9, plot=True, live=False, ghosts=Environment.hostile_ghosts)
agent.save()

100%|██████████| 100/100 [12:29<00:00,  7.49s/it]


<Figure size 432x288 with 0 Axes>

In [4]:
agent.epsilon = 0.1
agent.train(live=True, episodes=10)

 30%|███       | 3/10 [03:10<07:23, 63.41s/it] 


error: display Surface quit